## PySAL Change Log Statistics: Table Generation

This notebook generates the summary statistics for use in the 6-month releases of PySAL, which is now a meta package. 

It assumes the subpackages have been git cloned in a directory below the location of this notebook. It also requires network connectivity for some of the reporting.

Run this notebook after `100-gitcount.ipynb`


In [102]:
from __future__ import print_function
import os
import json
import re
import sys
import pandas
import subprocess
from subprocess import check_output

#import yaml
from datetime import datetime, timedelta

from dateutil.parser import parse
import pytz

utc=pytz.UTC

try:
    from urllib import urlopen
except:
    from urllib.request import urlopen




In [103]:
PYSAL_RELEASE = '2022-01-31' # date of this release
release_date = datetime.strptime(PYSAL_RELEASE+" 0:0:0", "%Y-%m-%d %H:%M:%S")

In [104]:
release_date

datetime.datetime(2022, 1, 31, 0, 0)

In [105]:
CWD = os.path.abspath(os.path.curdir)

In [106]:
CWD

'/home/serge/Documents/p/pysal/src/pysal/tools'

In [107]:
start_date = '2021-07-31' # date of previous meta release
since_date = '--since="{start}"'.format(start=start_date)
since_date
since = datetime.strptime(start_date+" 0:0:0", "%Y-%m-%d %H:%M:%S")
since

datetime.datetime(2021, 7, 31, 0, 0)

In [108]:
since_date

'--since="2021-07-31"'

In [109]:
with open('frozen.txt', 'r') as package_list:
    packages = package_list.readlines()
    packages = dict([package.strip().split(">=") for package in packages])

In [110]:
packages

{'libpysal': '4.6.0',
 'access': '1.1.3',
 'esda': '2.4.1',
 'giddy': '2.3.3',
 'inequality': '1.0.0',
 'pointpats': '2.2.0',
 'segregation': '2.1.0',
 'spaghetti': '1.6.5',
 'mgwr': '2.1.2',
 'momepy': '0.5.2',
 'spglm': '1.0.8',
 'spint': '1.0.7',
 'spreg': '1.2.4',
 'spvcm': '0.3.0',
 'tobler': '0.8.2',
 'mapclassify': '2.4.3',
 'splot': '1.1.4',
 'spopt': '0.2.1'}

import pysal
packages['pysal'] = pysal.__version__

In [111]:
import pickle

In [112]:
issues_closed = pickle.load(open("issues_closed.p", 'rb'))
pulls_closed = pickle.load(open('pulls_closed.p', 'rb'))

In [113]:
type(issues_closed)

dict

In [114]:
issues_closed.keys()

dict_keys(['libpysal', 'access', 'esda', 'giddy', 'inequality', 'pointpats', 'segregation', 'spaghetti', 'mgwr', 'momepy', 'spglm', 'spint', 'spreg', 'spvcm', 'tobler', 'mapclassify', 'splot', 'spopt', 'pysal'])

In [115]:
from release_info import get_pypi_info, get_github_info, clone_masters

In [116]:
#github_releases = get_github_info()

github_releases = pickle.load(open("releases.p", 'rb'))


In [117]:
pypi_releases = get_pypi_info()

In [118]:
from datetime import datetime

In [119]:
pysal_date = datetime.strptime('2021-07-31T12:00:00Z', '%Y-%m-%dT%H:%M:%SZ')
#ISO8601 = "%Y-%m-%dT%H:%M:%SZ"


In [120]:
pysal_rel = {'version': 'v2.5.0',
            'release_date': pysal_date}
github_releases['pysal'] = pysal_rel

In [121]:
github_releases

{'libpysal': {'version': 'v4.6.0',
  'url': 'https://api.github.com/repos/pysal/libpysal/tarball/v4.6.0',
  'release_date': datetime.datetime(2022, 1, 23, 0, 31, 33)},
 'access': {'version': 'V1.1.3',
  'url': 'https://api.github.com/repos/pysal/access/tarball/V1.1.3',
  'release_date': datetime.datetime(2021, 1, 31, 21, 31, 11)},
 'esda': {'version': 'v2.4.1',
  'url': 'https://api.github.com/repos/pysal/esda/tarball/v2.4.1',
  'release_date': datetime.datetime(2021, 7, 27, 12, 54, 27)},
 'giddy': {'version': 'v2.3.3',
  'url': 'https://api.github.com/repos/pysal/giddy/tarball/v2.3.3',
  'release_date': datetime.datetime(2020, 6, 10, 4, 59, 45)},
 'inequality': {'version': 'v1.0.0',
  'url': 'https://api.github.com/repos/pysal/inequality/tarball/v1.0.0',
  'release_date': datetime.datetime(2018, 10, 31, 22, 28, 18)},
 'pointpats': {'version': 'v2.2.0',
  'url': 'https://api.github.com/repos/pysal/pointpats/tarball/v2.2.0',
  'release_date': datetime.datetime(2020, 7, 27, 22, 17, 33)},

In [122]:
from datetime import datetime
datetime.fromtimestamp(0)
ISO8601 = "%Y-%m-%dT%H:%M:%SZ"


final_pulls = {}
final_issues = {}
for package in packages:
    filtered_issues = []
    filtered_pulls = []
    released = github_releases[package]['release_date']
    package_pulls = pulls_closed[package]
    package_issues = issues_closed[package]
    for issue in package_issues:
        #print(issue['number'], issue['title'], issue['closed_at'])
        closed = datetime.strptime(issue['closed_at'], ISO8601)
        if closed <= released and closed > since:
            filtered_issues.append(issue)
    final_issues[package] = filtered_issues
    for pull in package_pulls:
        #print(pull['number'], pull['title'], pull['closed_at'])
        closed = datetime.strptime(pull['closed_at'], ISO8601)
        if closed <= released and closed > since:
            filtered_pulls.append(pull)
    final_pulls[package] = filtered_pulls
    print(package, released, len(package_issues), len(filtered_issues), len(package_pulls),
         len(filtered_pulls))

libpysal 2022-01-23 00:31:33 25 25 0 0
access 2021-01-31 21:31:11 0 0 0 0
esda 2021-07-27 12:54:27 8 0 0 0
giddy 2020-06-10 04:59:45 0 0 0 0
inequality 2018-10-31 22:28:18 0 0 0 0
pointpats 2020-07-27 22:17:33 2 0 1 0
segregation 2021-08-09 22:05:57 10 4 0 0
spaghetti 2022-01-07 18:17:58 16 12 0 0
mgwr 2020-09-08 21:20:34 10 0 4 0
momepy 2022-01-06 11:11:33 46 36 0 0
spglm 2020-09-08 20:34:08 0 0 0 0
spint 2020-09-09 02:28:50 0 0 0 0
spreg 2021-06-29 19:21:48 6 0 2 0
spvcm 2020-02-02 19:42:39 0 0 0 0
tobler 2021-06-30 18:24:55 6 0 4 0
mapclassify 2021-07-27 03:06:32 5 0 4 0
splot 2021-07-27 15:04:13 7 0 1 0
spopt 2021-11-03 01:33:07 27 22 0 0


In [123]:
issue_details = final_issues
pull_details = final_pulls

In [124]:
packages

{'libpysal': '4.6.0',
 'access': '1.1.3',
 'esda': '2.4.1',
 'giddy': '2.3.3',
 'inequality': '1.0.0',
 'pointpats': '2.2.0',
 'segregation': '2.1.0',
 'spaghetti': '1.6.5',
 'mgwr': '2.1.2',
 'momepy': '0.5.2',
 'spglm': '1.0.8',
 'spint': '1.0.7',
 'spreg': '1.2.4',
 'spvcm': '0.3.0',
 'tobler': '0.8.2',
 'mapclassify': '2.4.3',
 'splot': '1.1.4',
 'spopt': '0.2.1'}

In [125]:
github_releases['pysal']['release_date'] = release_date

In [126]:
released

datetime.datetime(2021, 11, 3, 1, 33, 7)

In [127]:
packages.keys()

dict_keys(['libpysal', 'access', 'esda', 'giddy', 'inequality', 'pointpats', 'segregation', 'spaghetti', 'mgwr', 'momepy', 'spglm', 'spint', 'spreg', 'spvcm', 'tobler', 'mapclassify', 'splot', 'spopt'])

In [128]:
spvcm = packages['spvcm']

In [129]:
## skip packages not released since last meta release

In [130]:
for package in github_releases:
    if github_releases[package]['release_date']>since:
        print("new: ",package)
    else:
        print('old:', package)

new:  libpysal
old: access
old: esda
old: giddy
old: inequality
old: pointpats
new:  segregation
new:  spaghetti
old: mgwr
new:  momepy
old: spglm
old: spint
old: spreg
old: spvcm
old: tobler
old: mapclassify
old: splot
new:  spopt
new:  pysal


In [131]:
github_releases

{'libpysal': {'version': 'v4.6.0',
  'url': 'https://api.github.com/repos/pysal/libpysal/tarball/v4.6.0',
  'release_date': datetime.datetime(2022, 1, 23, 0, 31, 33)},
 'access': {'version': 'V1.1.3',
  'url': 'https://api.github.com/repos/pysal/access/tarball/V1.1.3',
  'release_date': datetime.datetime(2021, 1, 31, 21, 31, 11)},
 'esda': {'version': 'v2.4.1',
  'url': 'https://api.github.com/repos/pysal/esda/tarball/v2.4.1',
  'release_date': datetime.datetime(2021, 7, 27, 12, 54, 27)},
 'giddy': {'version': 'v2.3.3',
  'url': 'https://api.github.com/repos/pysal/giddy/tarball/v2.3.3',
  'release_date': datetime.datetime(2020, 6, 10, 4, 59, 45)},
 'inequality': {'version': 'v1.0.0',
  'url': 'https://api.github.com/repos/pysal/inequality/tarball/v1.0.0',
  'release_date': datetime.datetime(2018, 10, 31, 22, 28, 18)},
 'pointpats': {'version': 'v2.2.0',
  'url': 'https://api.github.com/repos/pysal/pointpats/tarball/v2.2.0',
  'release_date': datetime.datetime(2020, 7, 27, 22, 17, 33)},

In [132]:
# commits
cmd = ['git', 'log', '--oneline', since_date]

activity = {}
total_commits = 0
tag_dates = {}
ncommits_total = 0
for subpackage in packages:
    released = github_releases[subpackage]['release_date']
    tag_date = released.strftime("%Y-%m-%d")
    tag_dates[subpackage] = tag_date
    print(tag_date)
    #tag_date = tag_dates[subpackage]
    ncommits = 0
    if released > since:
        os.chdir(CWD)
        os.chdir('tmp/{subpackage}'.format(subpackage=subpackage))
        cmd_until = cmd + ['--until="{tag_date}"'.format(tag_date=tag_date)]
        ncommits = len(check_output(cmd_until).splitlines())
        ncommits_total = len(check_output(cmd).splitlines())
    print(subpackage, ncommits_total, ncommits, tag_date)
    total_commits += ncommits
    activity[subpackage] = ncommits

2022-01-23
libpysal 32 32 2022-01-23
2021-01-31
access 32 0 2021-01-31
2021-07-27
esda 32 0 2021-07-27
2020-06-10
giddy 32 0 2020-06-10
2018-10-31
inequality 32 0 2018-10-31
2020-07-27
pointpats 32 0 2020-07-27
2021-08-09
segregation 17 2 2021-08-09
2022-01-07
spaghetti 35 35 2022-01-07
2020-09-08
mgwr 35 0 2020-09-08
2022-01-06
momepy 35 35 2022-01-06
2020-09-08
spglm 35 0 2020-09-08
2020-09-09
spint 35 0 2020-09-09
2021-06-29
spreg 35 0 2021-06-29
2020-02-02
spvcm 35 0 2020-02-02
2021-06-30
tobler 35 0 2021-06-30
2021-07-27
mapclassify 35 0 2021-07-27
2021-07-27
splot 35 0 2021-07-27
2021-11-03
spopt 62 57 2021-11-03


In [133]:
activity

{'libpysal': 32,
 'access': 0,
 'esda': 0,
 'giddy': 0,
 'inequality': 0,
 'pointpats': 0,
 'segregation': 2,
 'spaghetti': 35,
 'mgwr': 0,
 'momepy': 35,
 'spglm': 0,
 'spint': 0,
 'spreg': 0,
 'spvcm': 0,
 'tobler': 0,
 'mapclassify': 0,
 'splot': 0,
 'spopt': 57}

In [134]:
subpackage

'spopt'

In [135]:
CWD

'/home/serge/Documents/p/pysal/src/pysal/tools'

In [136]:
# commits
cmd = ['git', 'log', '--oneline', since_date]

activity = {}
total_commits = 0
for subpackage in packages:
    ncommits = 0
    tag_date = tag_dates[subpackage]
    released = github_releases[subpackage]['release_date']
    if released > since:
        os.chdir(CWD)
        os.chdir('tmp/{subpackage}'.format(subpackage=subpackage))
        cmd_until = cmd + ['--until="{tag_date}"'.format(tag_date=tag_date)]
        ncommits = len(check_output(cmd_until).splitlines())
        print(ncommits)
        ncommits_total = len(check_output(cmd).splitlines())
        print(subpackage, ncommits_total, ncommits, tag_date)
    total_commits += ncommits
    activity[subpackage] = ncommits

32
libpysal 32 32 2022-01-23
2
segregation 17 2 2021-08-09
35
spaghetti 35 35 2022-01-07
35
momepy 35 35 2022-01-06
57
spopt 62 57 2021-11-03


In [137]:
since_date

'--since="2021-07-31"'

In [138]:
cmd_until

['git', 'log', '--oneline', '--since="2021-07-31"', '--until="2021-11-03"']

In [171]:
identities = {'Levi John Wolf': ('ljwolf', 'Levi John Wolf'),
              'Serge Rey': ('Serge Rey', 'Sergio Rey', 'sjsrey', 'serge'),
              'Wei Kang': ('Wei Kang', 'weikang9009'),
              'Dani Arribas-Bel': ('Dani Arribas-Bel', 'darribas'),
              'Antti Härkönen': ( 'antth', 'Antti Härkönen', 'Antti Härkönen', 'Antth'  ),
              'Juan C Duque': ('Juan C Duque', "Juan Duque"),
              'Renan Xavier Cortes': ('Renan Xavier Cortes', 'renanxcortes', 'Renan Xavier Cortes'   ),
              'Taylor Oshan': ('Tayloroshan', 'Taylor Oshan', 'TaylorOshan'),
              'Germano Barcelos': ('Germano Barcelos')
              'Tom Gertin': ('@Tomgertin', 'Tom Gertin', '@tomgertin')
}

def regularize_identity(string):
    string = string.decode()
    for name, aliases in identities.items():
        for alias in aliases:
            if alias in string:
                string = string.replace(alias, name)
    if len(string.split(' '))>1:
        string = string.title()
    return string.lstrip('* ')

In [172]:
author_cmd = ['git', 'log', '--format=* %aN', since_date]

In [173]:
author_cmd.append('blank')

In [174]:
author_cmd

['git', 'log', '--format=* %aN', '--since="2021-07-31"', 'blank']

In [175]:
from collections import Counter

In [176]:
tag_dates

{'libpysal': '2022-01-23',
 'access': '2021-01-31',
 'esda': '2021-07-27',
 'giddy': '2020-06-10',
 'inequality': '2018-10-31',
 'pointpats': '2020-07-27',
 'segregation': '2021-08-09',
 'spaghetti': '2022-01-07',
 'mgwr': '2020-09-08',
 'momepy': '2022-01-06',
 'spglm': '2020-09-08',
 'spint': '2020-09-09',
 'spreg': '2021-06-29',
 'spvcm': '2020-02-02',
 'tobler': '2021-06-30',
 'mapclassify': '2021-07-27',
 'splot': '2021-07-27',
 'spopt': '2021-11-03'}

In [177]:
authors_global = set()
authors = {}
global_counter = Counter()
counters = dict()
cmd = ['git', 'log', '--oneline', since_date]
total_commits = 0
activity = {}
for subpackage in packages:
    ncommits = 0
    released = github_releases[subpackage]['release_date']
    if released > since:
        os.chdir(CWD)
        os.chdir('tmp/{subpackage}'.format(subpackage=subpackage))
        ncommits = len(check_output(cmd).splitlines())
        print(cmd)
        tag_date = tag_dates[subpackage]
        tag_date = (datetime.strptime(tag_date, '%Y-%m-%d') + timedelta(days=1)).strftime('%Y-%m-%d')
        author_cmd[-1] = '--until="{tag_date}"'.format(tag_date=tag_date)
        #cmd_until = cmd + ['--until="{tag_date}"'.format(tag_date=tag_date)]
        print(subpackage, author_cmd)


        all_authors = check_output(author_cmd).splitlines()
        counter = Counter([regularize_identity(author) for author in all_authors])
        global_counter += counter
        counters.update({subpackage: counter})
        unique_authors = sorted(set(all_authors))
        authors[subpackage] =  unique_authors
        authors_global.update(unique_authors)
    total_commits += ncommits
    activity[subpackage] = ncommits

['git', 'log', '--oneline', '--since="2021-07-31"']
libpysal ['git', 'log', '--format=* %aN', '--since="2021-07-31"', '--until="2022-01-24"']
['git', 'log', '--oneline', '--since="2021-07-31"']
segregation ['git', 'log', '--format=* %aN', '--since="2021-07-31"', '--until="2021-08-10"']
['git', 'log', '--oneline', '--since="2021-07-31"']
spaghetti ['git', 'log', '--format=* %aN', '--since="2021-07-31"', '--until="2022-01-08"']
['git', 'log', '--oneline', '--since="2021-07-31"']
momepy ['git', 'log', '--format=* %aN', '--since="2021-07-31"', '--until="2022-01-07"']
['git', 'log', '--oneline', '--since="2021-07-31"']
spopt ['git', 'log', '--format=* %aN', '--since="2021-07-31"', '--until="2021-11-04"']


In [178]:
author_cmd

['git',
 'log',
 '--format=* %aN',
 '--since="2021-07-31"',
 '--until="2021-11-04"']

In [179]:
subpackage

'spopt'

In [180]:
counter

Counter({'James Gaboardi': 23,
         'Gegen07': 23,
         'Serge Rey': 2,
         'Germano Barcelos': 9})

In [149]:
authors_global

{b'* Charles Schmidt',
 b'* Elliott Sales de Andrade',
 b'* Germano Barcelos',
 b'* James Gaboardi',
 b'* Levi John Wolf',
 b'* Martin Fleischmann',
 b'* Serge Rey',
 b'* eli knaap',
 b'* gegen07',
 b'* ljwolf'}

In [150]:
activity

{'libpysal': 32,
 'access': 0,
 'esda': 0,
 'giddy': 0,
 'inequality': 0,
 'pointpats': 0,
 'segregation': 17,
 'spaghetti': 35,
 'mgwr': 0,
 'momepy': 35,
 'spglm': 0,
 'spint': 0,
 'spreg': 0,
 'spvcm': 0,
 'tobler': 0,
 'mapclassify': 0,
 'splot': 0,
 'spopt': 62}

In [151]:
counters

{'libpysal': Counter({'Lggermagermano Barceloso Barcelosrmagermano Barceloso Barcgermagermano Barceloso Barceloslosvi Johgermano Barcelos Wolf': 16,
          'Charlggermagermano Barceloso Barcelosrmagermano Barceloso Barcgermagermano Barceloso Barcelosloss Schmidt': 1,
          'Martigermano Barcelos Flggermagermano Barceloso Barcelosrmagermano Barceloso Barcgermagermano Barceloso Barceloslosischmagermano Barcelosgermano Barcelos': 3,
          'Jamggermagermano Barceloso Barcelosrmagermano Barceloso Barcgermagermano Barceloso Barcelosloss Gggermagermano Barceloso Barcelosrmagermano Barceloso Barcgermagermano Barceloso Barceloslosrmagermano Barceloso Barcggermagermano Barceloso Barcelosrmagermano Barceloso Barcgermagermano Barceloso Barcelosloslosaboardi': 7,
          'Elliott Salggermagermano Barceloso Barcelosrmagermano Barceloso Barcgermagermano Barceloso Barcelosloss Dggermagermano Barceloso Barcelosrmagermano Barceloso Barcgermagermano Barceloso Barceloslos Agermano Barcelosdra

In [152]:
counters

{'libpysal': Counter({'Lggermagermano Barceloso Barcelosrmagermano Barceloso Barcgermagermano Barceloso Barceloslosvi Johgermano Barcelos Wolf': 16,
          'Charlggermagermano Barceloso Barcelosrmagermano Barceloso Barcgermagermano Barceloso Barcelosloss Schmidt': 1,
          'Martigermano Barcelos Flggermagermano Barceloso Barcelosrmagermano Barceloso Barcgermagermano Barceloso Barceloslosischmagermano Barcelosgermano Barcelos': 3,
          'Jamggermagermano Barceloso Barcelosrmagermano Barceloso Barcgermagermano Barceloso Barcelosloss Gggermagermano Barceloso Barcelosrmagermano Barceloso Barcgermagermano Barceloso Barceloslosrmagermano Barceloso Barcggermagermano Barceloso Barcelosrmagermano Barceloso Barcgermagermano Barceloso Barcelosloslosaboardi': 7,
          'Elliott Salggermagermano Barceloso Barcelosrmagermano Barceloso Barcgermagermano Barceloso Barcelosloss Dggermagermano Barceloso Barcelosrmagermano Barceloso Barcgermagermano Barceloso Barceloslos Agermano Barcelosdra

In [153]:
def get_tag(title, level="##", as_string=True):
    words = title.split()
    tag = "-".join([word.lower() for word in words])
    heading = level+" "+title
    line = "\n\n<a name=\"{}\"></a>".format(tag)
    lines = [line]
    lines.append(heading)
    if as_string:
        return "\n".join(lines)
    else:
        return lines

In [154]:
subs = issue_details.keys()
table = []
txt = []
lines = get_tag("Changes by Package", as_string=False)

for sub in subs:
    total= issue_details[sub]
    pr = pull_details[sub]
    
    row = [sub, activity[sub], len(total), len(pr)]
    table.append(row)
    #line = "\n<a name=\"{sub}\"></a>".format(sub=sub)
    #lines.append(line)
    #line = "### {sub}".format(sub=sub)
    #lines.append(line)
    lines.extend(get_tag(sub.lower(), "###", as_string=False))
    for issue in total:
        url = issue['html_url']
        title = issue['title']
        number = issue['number']
        line = "* [#{number}:]({url}) {title} ".format(title=title,
                                                     number=number,
                                                     url=url)
        lines.append(line)



In [155]:
sub

'spopt'

In [156]:
os.chdir(CWD)

import pandas

In [157]:
df = pandas.DataFrame(table, columns=['package', 'commits', 'total issues', 'pulls'])

In [158]:
df.head()

,package,commits,total issues,pulls
0,libpysal,32,25,0
1,access,0,0,0
2,esda,0,0,0
3,giddy,0,0,0
4,inequality,0,0,0


In [159]:
df.sort_values(['commits','pulls'], ascending=False)\
  .to_html('./commit_table.html', index=None)

In [160]:
df.sum()

package         libpysalaccessesdagiddyinequalitypointpatssegr...
commits                                                       181
total issues                                                   99
pulls                                                           0
dtype: object

In [161]:
contributor_table = pandas.DataFrame.from_dict(counters).fillna(0).astype(int).T

In [162]:
contributor_table.to_html('./contributor_table.html')

In [163]:
totals = contributor_table.sum(axis=0).T
totals.sort_index().to_frame('commits')

,commits
Charlggermagermano Barceloso Barcelosrmagermano Barceloso Barcgermagermano Barceloso Barcelosloss Schmidt,1
Elliott Salggermagermano Barceloso Barcelosrmagermano Barceloso Barcgermagermano Barceloso Barcelosloss Dggermagermano Barceloso Barcelosrmagermano Barceloso Barcgermagermano Barceloso Barceloslos Agermano Barcelosdradggermagermano Barceloso Barcelosrmagermano Barceloso Barcgermagermano Barceloso Barceloslos,3
Ggermagermano Barceloso Barcelosrmagermano Barceloso Barcgermagermano Barceloso Barceloslosggermagermano Barceloso Barcelosrmagermano Barceloso Barcgermagermano Barceloso Barceloslosggermagermano Barceloso Barcelosrmagermano Barceloso Barcgermagermano Barceloso Barceloslosggermagermano Barceloso Barcelosrmagermano Barceloso Barcgermagermano Barceloso Barceloslosgermano Barcelosgermano Barcelosgermano Barcelos,23
Ggermagermano Barceloso Barcelosrmagermano Barceloso Barcgermagermano Barceloso Barceloslosli Kgermano Barcelosaap,8
Gggermagermano Barceloso Barcelosrmagermano Barceloso Barcgermagermano Barceloso Barceloslosrmagermano Barceloso Barcggermagermano Barceloso Barcelosrmagermano Barceloso Barcgermagermano Barceloso Barcelosloslosggermagermano Barceloso Barcelosrmagermano Barceloso Barcgermagermano Barceloso Barceloslosrmagermano Barceloso Barcggermagermano Barceloso Barcelosrmagermano Barceloso Barcgermagermano Barceloso Barcelosloslos,9
Jamggermagermano Barceloso Barcelosrmagermano Barceloso Barcgermagermano Barceloso Barcelosloss Gggermagermano Barceloso Barcelosrmagermano Barceloso Barcgermagermano Barceloso Barceloslosrmagermano Barceloso Barcggermagermano Barceloso Barcelosrmagermano Barceloso Barcgermagermano Barceloso Barcelosloslosaboardi,73
Lggermagermano Barceloso Barcelosrmagermano Barceloso Barcgermagermano Barceloso Barceloslosvi Johgermano Barcelos Wolf,16
Martigermano Barcelos Flggermagermano Barceloso Barcelosrmagermano Barceloso Barcgermagermano Barceloso Barceloslosischmagermano Barcelosgermano Barcelos,30
Sggermagermano Barceloso Barcelosrmagermano Barceloso Barcgermagermano Barceloso Barceloslosrggermagermano Barceloso Barcelosrmagermano Barceloso Barcgermagermano Barceloso Barceloslosggermagermano Barceloso Barcelosrmagermano Barceloso Barcgermagermano Barceloso Barceloslos Rggermagermano Barceloso Barcelosrmagermano Barceloso Barcgermagermano Barceloso Barceloslosy,2


In [164]:
totals = contributor_table.sum(axis=0).T
totals.sort_index().to_frame('commits').to_html('./commits_by_person.html')

In [165]:
totals

Lggermagermano Barceloso Barcelosrmagermano Barceloso Barcgermagermano Barceloso Barceloslosvi Johgermano Barcelos Wolf                                                                                                                                                                                                                                                                                                                  16
Charlggermagermano Barceloso Barcelosrmagermano Barceloso Barcgermagermano Barceloso Barcelosloss Schmidt                                                                                                                                                                                                                                                                                                                                 1
Martigermano Barcelos Flggermagermano Barceloso Barcelosrmagermano Barceloso Barcgermagermano Barceloso Barceloslosischmagermano Barcelosgermano

In [166]:
n_commits = df.commits.sum()
n_issues = df['total issues'].sum()
n_pulls = df.pulls.sum()

In [167]:
n_commits

181

In [168]:
#Overall, there were 719 commits that closed 240 issues, together with 105 pull requests across 12 packages since our last release on 2017-11-03.
#('{0} Here is a really long '
#           'sentence with {1}').format(3, 5))
line = ('Overall, there were {n_commits} commits that closed {n_issues} issues,'  
    ' together with {n_pulls} pull requests since our last release' 
        ' on {since_date}.\n'.format(n_commits=n_commits, n_issues=n_issues,
        n_pulls=n_pulls, since_date = start_date))

In [169]:
line

'Overall, there were 181 commits that closed 99 issues, together with 0 pull requests since our last release on 2021-07-31.\n'

## append html files to end of changes.md with tags for toc

In [170]:
with open('changes.md', 'w') as fout:
    fout.write(line)
    fout.write("\n".join(lines))
    fout.write(get_tag("Contributors"))
    fout.write("\n\nMany thanks to all of the following individuals who contributed to this release:\n\n")
    
    
    
    totals = contributor_table.sum(axis=0).T
    contributors = totals.index.values
    contributors.sort()
    contributors = contributors.tolist() 
    contributors = [ f'\n - {contributor}' for contributor in contributors]
    fout.write("".join(contributors))
    
